In [1]:
# Standard library imports.
import os
import requests
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Third party imports.
import pandas as pd
from tqdm import tqdm

In [2]:
class MELIData:
    def __init__(self) -> None:
        self.meli_url = "https://api.mercadolibre.com/"
        self.dataset_path = "../data/raw"

    def get_sites(self):
        url = f"{self.meli_url}/sites"
        request = requests.get(url)
        sites = request.json()
        return sites

    def get_categories(self, site_id):
        url = f"{self.meli_url}/sites/{site_id}/categories"
        request = requests.get(url)
        categories = request.json()
        return categories

    def search_item_by_category(self, site_id, cat_id, offset=50):
        url = f"{self.meli_url}/sites/{site_id}/search"
        params = {'category': cat_id, 'offset': offset}
        request = requests.get(url, params=params)
        items = request.json()
        return items

    def get_item_features(self, item_id):
        url = f"{self.meli_url}/items/{item_id}"
        request = requests.get(url)
        features = request.json()
        return features

    def convert_and_save_dataframe(self, arr, df_name):
        dataframe = pd.DataFrame(arr)
        dataframe.to_csv(f"{self.dataset_path}/{df_name}.csv", index=False)

    def read_df(self, df_name):
        dataframe = pd.read_csv(f"{self.dataset_path}/{df_name}.csv")
        return dataframe

In [3]:
melidataset = MELIData()

In [4]:
# Get sites - each country has its ID
sites = melidataset.get_sites()
print("sites:", type(sites))
print("sites[0]:", type(sites[0]))

sites: <class 'list'>
sites[0]: <class 'dict'>


In [5]:
melidataset.convert_and_save_dataframe(sites, "sites")

In [6]:
# Get categories - Categories of items belonging to the Marketplace
site_id = 'MCO'
categories = melidataset.get_categories(site_id)
print("categories:", type(categories))
print("categories[0]:", type(categories[0]))

categories: <class 'list'>
categories[0]: <class 'dict'>


In [7]:
melidataset.convert_and_save_dataframe(categories, "categories_MCO")

In [ ]:
# Get items
items = pd.DataFrame()
item_attributes = pd.DataFrame()
item_pictures = pd.DataFrame()

site_id = 'MCO'
df_name = f"categories_{site_id}"
categories = melidataset.read_df(df_name)

# Get the first 1000 items per search
# for each category in the marketplace.
offsets = list(range(0, 1000, 50))

for _, row in tqdm(categories.iterrows(), total=len(categories)):
    # print("key:", type(key))
    category_id = row['id']

    for offset in tqdm(offsets, total=len(offsets)):
        item_cats = melidataset.search_item_by_category(site_id, category_id, offset)
        if 'results' in item_cats:
            resulting_items = item_cats['results']
            if resulting_items is not None:
                for resulting_item in tqdm(resulting_items, total=len(resulting_items)):
                    # The content of 'shipping' is a dictionary.
                    # For this reason it is decided to take the keys and their values
                    # and add them directly to the content of the item
                    shipping_subdict = resulting_item.pop('shipping')
                    if shipping_subdict is not None:
                        resulting_item.update(shipping_subdict)

                    # The content of 'installments' is a dictionary.
                    # For this reason it is decided to take the keys and their values
                    # and add them directly to the content of the item
                    installments_subdict = resulting_item.pop('installments')
                    if installments_subdict is not None:
                        resulting_item.update(installments_subdict)

                    # Add the seller id directly to the item content.
                    resulting_item['seller_id'] = resulting_item['seller']['id']
                    resulting_item.pop('seller')

                    # The item attributes are a list of dictionaries.
                    # It is decided to save them as a separate file
                    attributes = resulting_item['attributes']
                    attributes_ = [{
                        'site_id': site_id,
                        'category_id': category_id,
                        'item_id': resulting_item['id'],
                        **attr,
                    } for attr in attributes]
                    resulting_item.pop('attributes')

                    # The features obtained from the items are not complete.
                    # For this reason, an additional query is made and the relevant keys are stored.
                    features = melidataset.get_item_features(resulting_item['id'])
                    resulting_item['initial_quantity'] = features['initial_quantity']
                    resulting_item['warranty'] = features['warranty']

                    # The 'pictures' are a list of dictionaries.
                    # It is decided to save them as a separate file
                    pictures = features['pictures']
                    pictures_ = [{
                        'site_id': site_id,
                        'category_id': category_id,
                        'item_id': resulting_item['id'],
                        **attr,
                    } for attr in pictures]

                    items_df = pd.DataFrame([resulting_item])
                    item_attributes_df = pd.DataFrame(attributes_)
                    item_pictures_df = pd.DataFrame(pictures_)
                
                    items = pd.concat([items, items_df], ignore_index=True)
                    item_attributes = pd.concat([item_attributes, item_attributes_df], ignore_index=True)
                    item_pictures = pd.concat([item_pictures, item_pictures_df], ignore_index=True)
                os.system('clear')

melidataset.convert_and_save_dataframe(items, "items_MCO")

item_attributes = item_attributes.drop(['values'], axis=1)
melidataset.convert_and_save_dataframe(item_attributes, "item_attributes_MCO")

melidataset.convert_and_save_dataframe(item_pictures, "item_pictures_MCO")

In [9]:
items = melidataset.read_df("items_MCO")
item_attributes = melidataset.read_df("item_attributes_MCO")
item_pictures = melidataset.read_df("item_pictures_MCO")

In [10]:
item_cols = list(items.columns)
print("Item cols:", item_cols, len(item_cols), end="\n\n")
items.head()

Item cols: ['id', 'title', 'condition', 'thumbnail_id', 'catalog_product_id', 'listing_type_id', 'permalink', 'buying_mode', 'site_id', 'category_id', 'domain_id', 'thumbnail', 'currency_id', 'order_backend', 'price', 'original_price', 'sale_price', 'available_quantity', 'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago', 'stop_time', 'winner_item_id', 'catalog_listing', 'discounts', 'promotions', 'inventory_id', 'store_pick_up', 'free_shipping', 'logistic_type', 'mode', 'tags', 'benefits', 'promise', 'quantity', 'amount', 'rate', 'seller_id', 'initial_quantity', 'warranty', 'differential_pricing', 'variation_filters', 'variations_data', 'official_store_name', 'location', 'seller_contact'] 46



,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,...,rate,seller_id,initial_quantity,warranty,differential_pricing,variation_filters,variations_data,official_store_name,location,seller_contact
0,MCO2196744036,Intercomunicador Auriculares Casco Bluetooh Bt...,new,929862-MLU73884050427_012024,MCO23892470,gold_special,https://www.mercadolibre.com.co/intercomunicad...,buy_it_now,MCO,MCO90069,...,0.0,170121574,999,Garantía del vendedor: 1 meses,NaN,NaN,NaN,NaN,NaN,NaN
1,MCO1403134263,Espejo Retrovisor Con Cámara Táctil 9.66 Inch ...,new,690567-MLU73062380373_112023,MCO28368900,gold_special,https://www.mercadolibre.com.co/espejo-retrovi...,buy_it_now,MCO,MCO177991,...,0.0,736375189,1500,Garantía del vendedor: 1 meses,NaN,NaN,NaN,NaN,NaN,NaN
2,MCO468073609,Limpiador De Inyectores De Motor A Gasolina Li...,new,789647-MCO53199125684_012023,NaN,gold_special,https://articulo.mercadolibre.com.co/MCO-46807...,buy_it_now,MCO,MCO166028,...,0.0,31508529,4126,Garantía de fábrica: 30 días,NaN,NaN,NaN,NaN,NaN,NaN
3,MCO1312378271,Llanta Hifly Hf201 P 185/60r14 82 H,new,876772-MLA74406430479_022024,MCO17976345,gold_special,https://www.mercadolibre.com.co/llanta-hifly-h...,buy_it_now,MCO,MCO23030,...,0.0,305902091,404,Garantía de fábrica: 5 años,NaN,NaN,NaN,NaN,NaN,NaN
4,MCO2260055390,Candado Con Alarma Para Motos Y Bicicletas,new,605009-MLU72418214106_102023,MCO27911986,gold_pro,https://www.mercadolibre.com.co/candado-con-al...,buy_it_now,MCO,MCO180780,...,0.0,263335107,500,Garantía del vendedor: 30 días,{'id': 33602181},NaN,NaN,NaN,NaN,NaN


In [11]:
# Delete columns that are not useful for the model
items.drop(['thumbnail_id', 'catalog_product_id', 'permalink', 'currency_id', 'order_backend', 'original_price',
                'sale_price', 'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago', 'winner_item_id',
                'catalog_listing', 'discounts', 'promotions', 'inventory_id', 'mode', 'tags', 'benefits', 'promise',
                'quantity', 'amount', 'rate', 'initial_quantity', 'differential_pricing', 'variation_filters',
                'variations_data', 'site_id', 'location', 'seller_contact'], axis=1, inplace=True)
item_cols = list(items.columns)
print("Item cols:", item_cols, len(item_cols), end="\n\n")
items.head()

Item cols: ['id', 'title', 'condition', 'listing_type_id', 'buying_mode', 'category_id', 'domain_id', 'thumbnail', 'price', 'available_quantity', 'stop_time', 'store_pick_up', 'free_shipping', 'logistic_type', 'seller_id', 'warranty', 'official_store_name'] 17



,id,title,condition,listing_type_id,buying_mode,category_id,domain_id,thumbnail,price,available_quantity,stop_time,store_pick_up,free_shipping,logistic_type,seller_id,warranty,official_store_name
0,MCO2196744036,Intercomunicador Auriculares Casco Bluetooh Bt...,new,gold_special,buy_it_now,MCO90069,MCO-MOTORCYCLE_INTERCOMMUNICATORS,http://http2.mlstatic.com/D_929862-MLU73884050...,54741.0,500,2044-01-30T04:00:00.000Z,False,False,cross_docking,170121574,Garantía del vendedor: 1 meses,NaN
1,MCO1403134263,Espejo Retrovisor Con Cámara Táctil 9.66 Inch ...,new,gold_special,buy_it_now,MCO177991,MCO-VEHICLE_CAMERAS,http://http2.mlstatic.com/D_690567-MLU73062380...,167100.0,500,2044-03-10T04:00:00.000Z,False,True,cross_docking,736375189,Garantía del vendedor: 1 meses,NaN
2,MCO468073609,Limpiador De Inyectores De Motor A Gasolina Li...,new,gold_special,buy_it_now,MCO166028,MCO-VEHICLE_FUEL_INJECTORS,http://http2.mlstatic.com/D_789647-MCO53199125...,42900.0,500,2040-07-12T22:08:56.000Z,False,False,cross_docking,31508529,Garantía de fábrica: 30 días,NaN
3,MCO1312378271,Llanta Hifly Hf201 P 185/60r14 82 H,new,gold_special,buy_it_now,MCO23030,MCO-AUTOMOTIVE_TIRES,http://http2.mlstatic.com/D_876772-MLA74406430...,148500.0,1,2043-07-02T04:00:00.000Z,False,True,cross_docking,305902091,Garantía de fábrica: 5 años,NaN
4,MCO2260055390,Candado Con Alarma Para Motos Y Bicicletas,new,gold_pro,buy_it_now,MCO180780,MCO-BICYCLE_AND_MOTORCYCLE_ALARMS,http://http2.mlstatic.com/D_605009-MLU72418214...,21018.0,250,2044-03-07T04:00:00.000Z,False,False,cross_docking,263335107,Garantía del vendedor: 30 días,NaN


In [12]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31215 entries, 0 to 31214
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   31215 non-null  object 
 1   title                31215 non-null  object 
 2   condition            30215 non-null  object 
 3   listing_type_id      31215 non-null  object 
 4   buying_mode          31215 non-null  object 
 5   category_id          31215 non-null  object 
 6   domain_id            31215 non-null  object 
 7   thumbnail            31214 non-null  object 
 8   price                30662 non-null  float64
 9   available_quantity   31215 non-null  int64  
 10  stop_time            31215 non-null  object 
 11  store_pick_up        31215 non-null  bool   
 12  free_shipping        31215 non-null  bool   
 13  logistic_type        28450 non-null  object 
 14  seller_id            31215 non-null  int64  
 15  warranty             26277 non-null 

In [13]:
# Encode Categorical Data
condition_dummies = pd.get_dummies(items['condition'], prefix='condition')
items = pd.concat([items, condition_dummies], axis=1)
items.drop('condition', axis=1, inplace=True)

listing_type_id_dummies = pd.get_dummies(items['listing_type_id'], prefix='listing_type_id')
items = pd.concat([items, listing_type_id_dummies], axis=1)
items.drop('listing_type_id', axis=1, inplace=True)

buying_mode_dummies = pd.get_dummies(items['buying_mode'], prefix='buying_mode')
items = pd.concat([items, buying_mode_dummies], axis=1)
items.drop('buying_mode', axis=1, inplace=True)

logistic_type_dummies = pd.get_dummies(items['logistic_type'], prefix='logistic_type')
items = pd.concat([items, logistic_type_dummies], axis=1)
items.drop('logistic_type', axis=1, inplace=True)

# Handle null values
items.fillna('', inplace=True)

# Convert stop_time column to datetime type
items['stop_time'] = pd.to_datetime(items['stop_time'])

In [14]:
# It'll not be used anymore
item_attribute_cols = list(item_attributes.columns)
print("Item attribute cols:", item_attribute_cols, len(item_attribute_cols), end="\n\n")
item_attributes.head()

Item attribute cols: ['site_id', 'category_id', 'item_id', 'id', 'name', 'value_id', 'value_name', 'attribute_group_id', 'attribute_group_name', 'value_struct', 'source', 'value_type'] 12



,site_id,category_id,item_id,id,name,value_id,value_name,attribute_group_id,attribute_group_name,value_struct,source,value_type
0,MCO,MCO1747,MCO2196744036,BRAND,Marca,19945578.0,Sikeo,OTHERS,Otros,NaN,1.000000e+00,string
1,MCO,MCO1747,MCO2196744036,ITEM_CONDITION,Condición del ítem,2230284.0,Nuevo,OTHERS,Otros,NaN,6.402815e+15,list
2,MCO,MCO1747,MCO2196744036,MODEL,Modelo,5856612.0,BT 12,OTHERS,Otros,NaN,1.000000e+00,string
3,MCO,MCO1747,MCO1403134263,BRAND,Marca,276243.0,Genérica,OTHERS,Otros,NaN,1.000000e+00,string
4,MCO,MCO1747,MCO1403134263,GTIN,Código universal de producto,NaN,7898513885022,OTHERS,Otros,NaN,6.402815e+15,string


In [15]:
# Delete columns that are not useful for the model
item_attributes.drop(['site_id', 'category_id', 'name', 'value_id', 'attribute_group_id', 'attribute_group_name', 'value_struct', 'source', 'value_type'], axis=1, inplace=True)
item_attribute_cols = list(item_attributes.columns)
print("Item attribute cols:", item_attribute_cols, len(item_attribute_cols), end="\n\n")
item_attributes.head()

Item attribute cols: ['item_id', 'id', 'value_name'] 3



,item_id,id,value_name
0,MCO2196744036,BRAND,Sikeo
1,MCO2196744036,ITEM_CONDITION,Nuevo
2,MCO2196744036,MODEL,BT 12
3,MCO1403134263,BRAND,Genérica
4,MCO1403134263,GTIN,7898513885022


In [16]:
item_picture_cols = list(item_pictures.columns)
print("Item picture cols:", item_picture_cols, len(item_picture_cols), end="\n\n")
item_pictures.head()

Item picture cols: ['site_id', 'category_id', 'item_id', 'id', 'url', 'secure_url', 'size', 'max_size', 'quality'] 9



,site_id,category_id,item_id,id,url,secure_url,size,max_size,quality
0,MCO,MCO1747,MCO2196744036,929862-MLU73884050427_012024,http://http2.mlstatic.com/D_929862-MLU73884050...,https://http2.mlstatic.com/D_929862-MLU7388405...,500x475,545x518,NaN
1,MCO,MCO1747,MCO2196744036,876934-MLU73883690363_012024,http://http2.mlstatic.com/D_876934-MLU73883690...,https://http2.mlstatic.com/D_876934-MLU7388369...,377x500,396x524,NaN
2,MCO,MCO1747,MCO2196744036,641308-MLU73882922341_012024,http://http2.mlstatic.com/D_641308-MLU73882922...,https://http2.mlstatic.com/D_641308-MLU7388292...,487x500,502x515,NaN
3,MCO,MCO1747,MCO2196744036,961114-MLU72761385682_112023,http://http2.mlstatic.com/D_961114-MLU72761385...,https://http2.mlstatic.com/D_961114-MLU7276138...,500x430,581x500,NaN
4,MCO,MCO1747,MCO2196744036,792271-MLU73886968385_012024,http://http2.mlstatic.com/D_792271-MLU73886968...,https://http2.mlstatic.com/D_792271-MLU7388696...,496x500,519x523,NaN


In [17]:
# Delete columns that are not useful for the model
item_pictures.drop(['site_id', 'category_id', 'secure_url', 'size', 'max_size', 'quality'], axis=1, inplace=True)
item_picture_cols = list(item_pictures.columns)
print("Item picture cols:", item_picture_cols, len(item_picture_cols), end="\n\n")
item_pictures.head()

Item picture cols: ['item_id', 'id', 'url'] 3



,item_id,id,url
0,MCO2196744036,929862-MLU73884050427_012024,http://http2.mlstatic.com/D_929862-MLU73884050...
1,MCO2196744036,876934-MLU73883690363_012024,http://http2.mlstatic.com/D_876934-MLU73883690...
2,MCO2196744036,641308-MLU73882922341_012024,http://http2.mlstatic.com/D_641308-MLU73882922...
3,MCO2196744036,961114-MLU72761385682_112023,http://http2.mlstatic.com/D_961114-MLU72761385...
4,MCO2196744036,792271-MLU73886968385_012024,http://http2.mlstatic.com/D_792271-MLU73886968...


In [18]:
dataset_path = "../data/processed"
def convert_and_save_dataframe(arr, df_name):
    dataframe = pd.DataFrame(arr)
    dataframe.to_csv(f"{dataset_path}/{df_name}.csv", index=False)

In [19]:
convert_and_save_dataframe(items, "items")
convert_and_save_dataframe(item_attributes, "item_attributes")
convert_and_save_dataframe(item_pictures, "item_pictures")